# SEIRX model runs for test technology and screening frequency investigation

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [2]:
# paths for I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [3]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    test_type,index_case,e_screen_interval,r_screen_interval,N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, test_type=test_type,
                index_case=index_case, e_screen_interval=e_screen_interval,
                r_screen_interval=r_screen_interval)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results,index_case,test_type=test_type,
                                e_screen_interval=e_screen_interval,
                                r_screen_interval=r_screen_interval)

    return row

## Screening parameters

In [4]:
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'same_day_LAMP', 'same_day_PCR',\
              'one_day_PCR', 'two_day_PCR']

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# resident and employee streening intervals (in days)
e_screen_range = [2, 3, 7, None]
r_screen_range = [2, 3, 7, None]

# ensemble_size
N_runs = 5000

screening_params = [(i, j, k, l, N_runs)\
              for i in test_types \
              for j in index_cases \
              for k in e_screen_range \
              for l in r_screen_range]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 160 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [5]:
with open('params/testing_strategy_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_strategy_measures.json') as json_file:
    measures = json.load(json_file)


In [6]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [7]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': 'same_day_antigen',
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [8]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/testing_strategy/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_testing_strategy_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 160/160 [2:03:18<00:00, 46.24s/it]  


CPU times: user 3.6 s, sys: 336 ms, total: 3.94 s
Wall time: 2h 3min 20s


,test_type,index_case,resident_screen_interval,employee_screen_interval,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,2.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.625720,0.0,0.0,0.0,0.0,0.0,1.0,0.0922,0.0,0.315783
1,same_day_antigen,employee,3.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,2.105632,0.0,0.0,0.0,0.0,0.0,1.0,0.1018,0.0,0.336837
2,same_day_antigen,employee,2.0,3.0,53.0,53.0,53.0,53.0,53.0,53.0,...,2.014842,0.0,0.0,0.0,0.0,1.0,1.0,0.1130,0.0,0.343882


## Wild type

In [9]:
ensmbl_dst = '../data/simulation_results/ensembles/vaccination/wild_type'

### Simulation parameters and measures in place

In [10]:
with open('params/testing_strategy_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_strategy_measures.json') as json_file:
    measures = json.load(json_file)


In [11]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.073033,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [12]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': 'same_day_antigen',
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [13]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 4
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_testing_strategy_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 160/160 [1:41:18<00:00, 37.99s/it]


CPU times: user 3.55 s, sys: 267 ms, total: 3.81 s
Wall time: 1h 41min 20s


,test_type,index_case,resident_screen_interval,employee_screen_interval,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,2.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,0.945220,0.0,0.0,0.0,0.0,0.0,1.0,0.0742,0.0,0.273330
1,same_day_antigen,employee,2.0,3.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.137260,0.0,0.0,0.0,0.0,0.0,1.0,0.0888,0.0,0.301551
2,same_day_antigen,employee,3.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.170706,0.0,0.0,0.0,0.0,0.0,1.0,0.0728,0.0,0.272608


## B.1.1.7 & employee masks

### Simulation parameters and measures in place

In [14]:
with open('params/testing_strategy_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_strategy_masks_measures.json') as json_file:
    measures = json.load(json_file)


In [15]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [16]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': 'same_day_antigen',
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': True,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [17]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/testing_strategy/masks'

number_of_cores = psutil.cpu_count(logical=True) - 2
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_testing_strategy_masks_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 160/160 [1:42:11<00:00, 38.32s/it]


CPU times: user 3.4 s, sys: 398 ms, total: 3.8 s
Wall time: 1h 42min 13s


,test_type,index_case,resident_screen_interval,employee_screen_interval,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,3.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.210921,0.0,0.0,0.0,0.0,0.0,1.0,0.0756,0.0,0.274043
1,same_day_antigen,employee,2.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,0.885520,0.0,0.0,0.0,0.0,0.0,1.0,0.0644,0.0,0.254294
2,same_day_antigen,employee,7.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.826512,0.0,0.0,0.0,0.0,0.0,1.0,0.0834,0.0,0.311872
